In [1]:
import pandas as pd
df = pd.read_csv("train.corpus.csv",sep='|')
df.head(100)

,id,folder,signer,annotation
0,01April_2010_Thursday_heute_default-0,01April_2010_Thursday_heute_default-0/1/*.png,Signer04,__ON__ LIEB ZUSCHAUER ABEND WINTER GESTERN loc...
1,01April_2010_Thursday_heute_default-2,01April_2010_Thursday_heute_default-2/1/*.png,Signer04,loc-WEST WARM loc-WEST BLEIBEN KUEHL
2,01April_2010_Thursday_heute_default-3,01April_2010_Thursday_heute_default-3/1/*.png,Signer04,KUEHL KUEHL IN-KOMMEND NACHT MINUS loc-MINUS-P...
3,01April_2010_Thursday_heute_default-4,01April_2010_Thursday_heute_default-4/1/*.png,Signer04,MORGEN TEMPERATUR ACHT BIS DREIZEHN MAXIMAL DR...
4,01April_2010_Thursday_heute_default-6,01April_2010_Thursday_heute_default-6/1/*.png,Signer04,AUCH SAMSTAG WEST FREUNDLICH IX TROCKEN WEST R...
...,...,...,...,...
95,01June_2010_Tuesday_tagesschau_default-2,01June_2010_Tuesday_tagesschau_default-2/1/*.png,Signer01,__ON__ MITTE EUROPA TIEF KOMMEN KOMMEN REGEN-P...
96,01June_2010_Tuesday_tagesschau_default-4,01June_2010_Tuesday_tagesschau_default-4/1/*.png,Signer01,__ON__ GLEICH ZEIT WEST KOMMEN HOCH KOMMEN __O...
97,01June_2010_Tuesday_tagesschau_default-6,01June_2010_Tuesday_tagesschau_default-6/1/*.png,Signer01,__ON__ TAG loc-SUEDOST REGEN-PLUSPLUS KLAR REG...
98,01June_2010_Tuesday_tagesschau_default-12,01June_2010_Tuesday_tagesschau_default-12/1/*.png,Signer01,__ON__ TAG ZWEIZWANZIG loc-REGION ZEHN ACHT RE...


In [5]:
import pandas as pd
import spacy
from collections import Counter
import time

In [11]:
def build_dictionary(files):
    start = time.time()
    lang_model = spacy.load('de_core_news_sm')
    end = time.time()
    print('load lang_model cost %.3f s'%(end-start))
    train = []
    # 合并annotation中的语料
    for file in files:
        df = pd.read_csv(file,sep='|')
        for i in range(len(df)):
            train.append(df.loc[i]['annotation'])

    # Create a dictionary which maps tokens to indices (train contains all the training sentences)
    freq_list = Counter()
    punctuation = ['_','NULL','ON','OFF','EMOTION','LEFTHAND','IX','PU']
    for sentence in train:
        sentence = [tok.text for tok in lang_model.tokenizer(sentence) if not tok.text in punctuation]
        freq_list.update(sentence)

    # 按照词的出现频率建立词典，词频越高索引越靠前
    freq_list = sorted(freq_list.items(),key=lambda item:item[1],reverse=True)
    dictionary = {}
    dictionary['<bos>'] = 0
    dictionary['<eos>'] = 1
    for i,item in enumerate(freq_list):
        dictionary[item[0]] = i+2
    print("Build dictionary successfully!")
    return dictionary

In [12]:
train_file = "train.corpus.csv"
dev_file = "dev.corpus.csv"
dictionary = build_dictionary([train_file,dev_file])

load lang_model cost 1.337 s
Build dictionary successfully!


In [13]:
dictionary

{'<bos>': 0,
 '<eos>': 1,
 'REGEN': 2,
 'MORGEN': 3,
 'WOLKE': 4,
 'SONNE': 5,
 'GRAD': 6,
 'REGION': 7,
 'WEHEN': 8,
 'SCHNEE': 9,
 'loc-REGION': 10,
 'NACHT': 11,
 'PU': 12,
 'GEWITTER': 13,
 'KOENNEN': 14,
 'MEHR': 15,
 'KOMMEN': 16,
 'AUCH': 17,
 'BISSCHEN': 18,
 'HEUTE': 19,
 'NORD': 20,
 'SCHAUER': 21,
 'WETTER': 22,
 'BIS': 23,
 'SUED': 24,
 'cl-KOMMEN': 25,
 'WIND': 26,
 'DANN': 27,
 'ZWANZIG': 28,
 'OST': 29,
 'NEBEL': 30,
 'MOEGLICH': 31,
 'ABEND': 32,
 'FREUNDLICH': 33,
 'JETZT': 34,
 'BERG': 35,
 'STURM': 36,
 'TEIL': 37,
 'ABER': 38,
 'WEST': 39,
 'MAESSIG': 40,
 'TAG': 41,
 'WECHSELHAFT': 42,
 'FLUSS': 43,
 'TIEF': 44,
 'WIE-AUSSEHEN': 45,
 'MINUS': 46,
 'SCHWACH': 47,
 'MITTE': 48,
 'SONNTAG': 49,
 'BLEIBEN': 50,
 'KUEHL': 51,
 'FREITAG': 52,
 'KLAR': 53,
 'HOCH': 54,
 'VIEL': 55,
 'TROCKEN': 56,
 'NORDRAUM': 57,
 'SAMSTAG': 58,
 'STARK': 59,
 'BESONDERS': 60,
 'DONNERSTAG': 61,
 'MEISTENS': 62,
 'TEMPERATUR': 63,
 'SONST': 64,
 'loc-NORD': 65,
 'KALT': 66,
 'IN-KOMMEND'